# Grape object detection - inference



Este notebook é utilizado para detectar cacho de uva em uma imagem a partir de uma modelo pré-treinado.

# Setup

### Install

In [ ]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

In [ ]:
!pip install pycocotools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

In [5]:
%%bash
cd models/research
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd models/research
pip install .

### Imports

In [23]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
from PIL import Image
import itertools
import matplotlib.patches as patches
from random import randrange
import math

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
#import matplotlib.image as mpimg 
#from matplotlib.pyplot import imshow
#%matplotlib inline
from PIL import Image
from IPython.display import display

Import the object detection module.

In [ ]:
%cd /content/drive/My Drive/Prova_pratica-processo_seletivo-pesquisadorI/Object_detection/models/research
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [9]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [10]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [11]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/drive/My Drive/Prova_pratica-processo_seletivo-pesquisadorI/Object_detection/models/research/object_detection/images/labelmap.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [12]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/drive/My Drive/Prova_pratica-processo_seletivo-pesquisadorI/Object_detection/models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS


[PosixPath('/content/drive/My Drive/Prova_pratica-processo_seletivo-pesquisadorI/Object_detection/models/research/object_detection/test_images/SVB_20180427_151905850_HDR.jpg')]

# Detection

Load an object detection model:

In [13]:
detection_model = tf.saved_model.load('/content/drive/My Drive/Prova_pratica-processo_seletivo-pesquisadorI/Object_detection/models/research/object_detection/inference_graph/saved_model')


Check the model's input signature, it expects a batch of 3-color images of type uint8:

In [ ]:
print(detection_model.signatures['serving_default'].inputs)

And returns several outputs:

In [ ]:
detection_model.signatures['serving_default'].output_dtypes

In [ ]:
detection_model.signatures['serving_default'].output_shapes

Add a wrapper function to call the model, and cleanup the outputs:

In [17]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [18]:
def filter_boxes(boxes, scores, classes, categories):
  """Return boxes with a confidence >= `min_score`"""
  n = len(classes)
  idxs = []
  for i in range(n):
      idxs.append(i)
  
  filtered_boxes = boxes[idxs, ...]
  filtered_scores = scores[idxs, ...]
  filtered_classes = classes[idxs, ...]
  return filtered_boxes, filtered_scores, filtered_classes

def calculate_coord(bbox, width, height):
  """Return boxes coordinates"""
  xmin = bbox[1] * width
  ymin = bbox[0] * height
  xmax = bbox[3] * width
  ymax = bbox[2] * height

  return [xmin, ymin, xmax - xmin, ymax - ymin]
    
def calculate_centr(coord):
  """Calculate centroid for each box"""
  return (coord[0]+(coord[2]/2), coord[1]+(coord[3]/2))
  
def calculate_centr_distances(centroid_1, centroid_2):
  """Calculate the distance between 2 centroids"""
  return  math.sqrt((centroid_2[0]-centroid_1[0])**2 + (centroid_2[1]-centroid_1[1])**2)
  
def calculate_perm(centroids):
  """Return all combinations of centroids"""
  permutations = []
  for current_permutation in itertools.permutations(centroids, 2):
    if current_permutation[::-1] not in permutations:
      permutations.append(current_permutation)
  return permutations
  
def midpoint(p1, p2):
  """Midpoint between 2 points"""
  return ((p1[0] + p2[0])/2, (p1[1] + p2[1])/2)
    
def calculate_slope(x1, y1, x2, y2):
  """Calculate slope"""
  m = (y2-y1)/(x2-x1)
  return m

Run it on each test image and show the results:

In [21]:
def show_inference(model, image_path):
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)

  # Get boxes 
  boxes, scores, classes = filter_boxes(output_dict['detection_boxes'], output_dict['detection_scores'], output_dict['detection_classes'], [1])

  # Get width and heigth
  im = Image.fromarray(image_np)
  width, height = im.size

  # Calculate normalized coordinates for boxes
  centroids = []
  coordinates = []
  for box in boxes:
    coord = calculate_coord(box, width, height)
    centr = calculate_centr(coord)
    centroids.append(centr)
    coordinates.append(coord)

  # Calculate all permutations
  permutations = calculate_perm(centroids)

  # Display boxes and centroids
  fig, ax = plt.subplots(figsize = (20,12), dpi = 90)
  ax.imshow(image, interpolation='nearest')
  for coord, centr in zip(coordinates, centroids):
    ax.add_patch(patches.Rectangle((coord[0], coord[1]), coord[2], coord[3], linewidth=2, edgecolor='y', facecolor='none', zorder=10))
    ax.add_patch(patches.Circle((centr[0], centr[1]), 3, color='yellow', zorder=20))

  # Display lines between centroids
  for perm in permutations:
    dist = calculate_centr_distances(perm[0], perm[1])
    dist_m = dist

    print("Distance[pixel]: ", dist_m)
    middle = midpoint(perm[0], perm[1])
    print("Middle point", middle)

    x1 = perm[0][0]
    x2 = perm[1][0]
    y1 = perm[0][1]
    y2 = perm[1][1]

    slope = calculate_slope(x1, y1, x2, y2)
    dy = math.sqrt(3**2/(slope**2+1))
    dx = -slope*dy

    # Display randomly the position of our distance text
    if randrange(10) % 2== 0:
      Dx = middle[0] - dx*10
      Dy = middle[1] - dy*10
    else:
      Dx = middle[0] + dx*10
      Dy = middle[1] + dy*10

    ax.annotate("{}px".format(round(dist_m, 2)), xy=middle, color='white', xytext=(Dx, Dy), fontsize=10, arrowprops=dict(arrowstyle='->', lw=1.5, color='yellow'), bbox=dict(facecolor='red', edgecolor='white', boxstyle='round', pad=0.2), zorder=30)
    ax.plot((perm[0][0], perm[1][0]), (perm[0][1], perm[1][1]), linewidth=2, color='yellow', zorder=15)
    
    

In [ ]:
# Make prediction
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)